# Plastic Sorting #

## Casptone ##

## Sección 1: Configuración y Preparación del Entorno ##

In [ ]:
# Sección 1: Configuración y Preparación del Entorno
import os
import json
import time
import random
import logging
import paho.mqtt.client as mqtt
from datetime import datetime

# Configurar logging
def setup_logger():
    logging.basicConfig(
        level=logging.INFO,
        format="[%(asctime)s] [%(levelname)s] %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S"
    )
setup_logger()
logging.info("[SETUP] Logger configurado correctamente.")

# Configuración del cliente MQTT
mqtt_config = {
    "broker_address": "localhost",  # Cambiar si se usa un broker remoto
    "port": 1883,
    "topics": {
        "sensor_data": "material/entrada"
    }
}

client = mqtt.Client("raspberry1")

# Conectar al broker MQTT
def connect_mqtt():
    try:
        client.connect(mqtt_config["broker_address"], mqtt_config["port"])
        logging.info(f"[MQTT] Conectado al broker {mqtt_config['broker_address']}:{mqtt_config['port']}")
    except Exception as e:
        logging.error(f"[MQTT] Error conectando al broker: {e}")

connect_mqtt()

# Callback para mensajes recibidos
def on_message(client, userdata, msg):
    logging.info(f"[MQTT] Mensaje recibido en {msg.topic}: {msg.payload.decode()}")

client.on_message = on_message
client.loop_start()

### Sección 2: Simulación de Lectura de Sensores (AS7265x)

In [5]:
# Sección 2: Simulación de Lectura de Sensores (AS7265x)
def simulate_sensor_reading():
    spectral_data = {f"channel_{i}": random.uniform(0.1, 1.0) for i in range(18)}
    detected_material = random.choice(["PET", "HDPE"])
    confidence = round(random.uniform(0.8, 1.0), 2)
    
    sensor_data = {
        "timestamp": datetime.now().isoformat(),
        "sensor_id": "AS7265x_1",
        "spectral_data": spectral_data,
        "detected_material": detected_material,
        "confidence": confidence
    }
    return sensor_data


### Sección 3: Publicación de Datos MQTT ###

In [4]:
# Sección 3: Publicación de Datos MQTT
def publish_sensor_data():
    try:
        data = simulate_sensor_reading()
        client.publish(mqtt_config["topics"]["sensor_data"], json.dumps(data))
        logging.info(f"[MQTT] Datos publicados: {data}")
    except Exception as e:
        logging.error(f"[MQTT] Error publicando datos: {e}")

#### Ejecución Continua ###

In [ ]:
# Ejecución Continua
if __name__ == "__main__":
    logging.info("[MAIN] Iniciando simulación de Raspberry Pi 1.")
    try:
        while True:
            publish_sensor_data()
            time.sleep(5)  # Publicar cada 5 segundos
    except KeyboardInterrupt:
        logging.info("[MAIN] Simulación detenida por el usuario.")
        client.loop_stop()
